In [ ]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal


In [ ]:
import aiida
aiida.load_profile()


In [ ]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


In [ ]:
# nodebank settings

from tools.nodebank import NodeBank
pwd = os.getcwd()
print(pwd)
nodebank = NodeBank(pwd)


In [ ]:
!pwd

In [ ]:
# init_struc_data, opt_struc_data, stat, rslt_data, ea_id_data, ea_data = cryspy_init.initialize()

In [ ]:
# init_struc_data

In [ ]:
import os
import numpy as np


from aiida.plugins import DataFactory
from aiida.orm import Code
from aiida.orm import Str, Dict, List, Int
from aiida.engine import calcfunction, WorkChain

from CrySPY.IO import read_input as rin
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')

# PandasFrameData = DataFactory('cryspy.dataframe')
PandasFrameData = DataFactory('dataframe.frame')

ConfigparserData = DataFactory('cryspy.configparser')
StructurecollectionData = DataFactory('cryspy.structurecollection')
EAData = DataFactory('cryspy.ea_data')
EAidData = DataFactory('cryspy.ea_id_data')


In [ ]:
# struc = StructurecollectionData(init_struc_data)
# struc.structurecollection

In [ ]:
pwd = os.getcwd()
cryspy_in = os.path.join(pwd, 'cryspy.in0')
inputs = {'cryspy_in': Str(cryspy_in)}
initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
initial_result = run(initialize_WorkChain, **inputs)


In [ ]:
# workflow結果表示
initial_result

In [ ]:
initial_result["rslt_data"].df

In [ ]:
import shutil
# cryspy.inは実行中に書き換えられる。
shutil.copy("cryspy.in0", "cryspy.in")


In [ ]:
@calcfunction
def generate_work_path(ea_id_node, structure_node):
    """make string of working directoryd from ea_id_node.id_queueing.
    """
    ea_id_data = ea_id_node.ea_id_data
    structures = structure_node.structurecollection
    id_queueing = ea_id_data[1]
    work_path_dic = {}
    structures_dic = {}
    for cid in id_queueing:
        work_path = './work/{:06}/'.format(cid)
        work_path = os.path.abspath(work_path)
        structures_dic[cid] = structures[cid]
        
        ID = str(cid)
        work_path_dic[ID] = work_path
    return Dict(dict=work_path_dic)

@calcfunction
def generate_structures(ea_id_node, structure_node):
    """make string of working directoryd from ea_id_node.id_queueing.
    """
    ea_id_data = ea_id_node.ea_id_data
    structures = structure_node.structurecollection
    id_queueing = ea_id_data[1]
    work_path_dic = {}
    structures_dic = {}
    for cid in id_queueing:
        work_path = './work/{:06}/'.format(cid)
        work_path = os.path.abspath(work_path)
        structures_dic[cid] = structures[cid]
        
        ID = str(cid)
        work_path_dic[ID] = work_path
    return StructurecollectionData(structures_dic)

ea_id_node = initial_result["ea_id_data"]
struc_node = initial_result["init_struc"]

work_path_node = generate_work_path(ea_id_node, struc_node)
initial_structures_node = generate_structures(ea_id_node, struc_node)

In [ ]:
initial_structures_node

# restart

# handle_jobs

In [ ]:
cwd_node = work_path_node
cwd_node.get_dict()

In [ ]:
computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label


In [ ]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes


In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [ ]:
code_lammps_opt

In [ ]:
if len(initial_structures_node.structurecollection.keys())>0 and len(cwd_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": initial_structures_node, 
             "cwd": cwd_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

In [ ]:
optimize_result["results"]

In [ ]:
opt_struc_node = optimize_result["final_structures"]
opt_struc_node.structurecollection

In [ ]:
ea_id_node.ea_id_data

In [ ]:
@calcfunction
def update_ea_id_data(ea_id_node, opt_results):
    """update ea_id_data.
    
    finished jobs are found in opt_results.
    """
    opt_results = opt_results.get_dict()
    
    gen, id_queueing, id_running = ea_id_node.ea_id_data
    for i in opt_results["index"]:
        if i in id_queueing:
            id_queueing.remove(i)
        else:
            raise ValueError('inconsistent queue and execution ID.')
        
        id_running.append(i)

    ea_id_data = (gen, id_queueing, id_running)
    return EAidData(ea_id_data)

opt_results_node = optimize_result["results"]
ea_id_node_new = update_ea_id_data(ea_id_node, opt_results_node)
ea_id_node_new.ea_id_data


In [ ]:
ea_id_node_new.get_dict()

In [ ]:
# ea_id_node = ea_id_node_new

In [ ]:
initial_structures_node

# next run

In [ ]:
stat, struc_dict = cryspy_restart.restart(initial_structures_node.structurecollection)


In [ ]:
struc_dict.keys()

In [ ]:
rslt_data_node = initial_result["rslt_data"]
rslt_data_node

In [ ]:
ea_id_data_node = initial_result["ea_id_data"]

In [ ]:
# optの対称性の登録
@calcfunction
def generate_rlst(initial_structures_node: StructurecollectionData, 
                  opt_struc_node: StructurecollectionData, 
                  optimize_result: Dict, 
                  ea_id_data_node: EAidData, 
                  rslt_data_node: PandasFrameData):
    init_struc_data = initial_structures_node.structurecollection
    opt_struc = opt_struc_node.structurecollection
    opt_results = optimize_result.get_dict()
    rslt_data = rslt_data_node.df
    gen = ea_id_data_node.ea_id_data[0]
    magmon = None
    check_opt = 'done'
    
    symprec = rin.symprec
    for i,energy in zip(opt_results["index"], opt_results["energy"]):
        if energy is not None:
            check_opt = 'done'
        else:
            check_opt = 'not_yet'
        if energy is None:
            energy = np.nan
        else:
            natot = len(init_struc_data[i].atomic_numbers)
            energy /= natot
            
        in_spg = init_struc_data[i].get_space_group_info(symprec=symprec)
        opt_spg = opt_struc[i].get_space_group_info(symprec=symprec)
        rslt_data.loc[i] = [gen, in_spg[1], in_spg[0], opt_spg[1], opt_spg[0], 
                           energy, magmon, check_opt]
        
    return PandasFrameData(rslt_data)
    # return rslt_data

rslt_data_node_new = generate_rlst(initial_structures_node, opt_struc_node, 
                              optimize_result["results"], 
                              ea_id_data_node, rslt_data_node)
rslt_data_node_new.df

# next_sg


In [ ]:
ea_data_node = initial_result["ea_data"]
ea_data_node.ea_data



In [ ]:
from CrySPY.IO import io_stat


In [ ]:
rslt_data_node = rslt_data_node_new
rslt_data_node.df

In [ ]:
opt_struc_node.structurecollection

In [ ]:
inputs = {'initial_structures': initial_structures_node,
         'optimized_structures': opt_struc_node,
          'rslt_data': rslt_data_node,
          'ea_id_data': ea_id_data_node,
          'ea_data': ea_data_node}
next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)

In [ ]:
next_sg_results

In [ ]:
next_sg_results["ea_id_data"].ea_id_data

In [ ]:
next_sg_results["ea_data"].ea_data

In [ ]:
next_sg_results["rslt_data"].df

In [ ]:
raise

# next run

In [ ]:
init_struc_data.keys()

In [ ]:
stat, struc_dict = cryspy_restart.restart()
struc_dict

In [ ]:
jobs = Ctrl_job(stat, struc_dict) # init_struc_dataの更新の必要がある。
jobs.check_job()

In [ ]:
stat["basic"]["tot_struc"]

In [ ]:
cwd_dict, struc_dict = handle_job_return_dict(jobs)

In [ ]:
jobs.init_struc_data

In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
cwd_dict, struc_dict


In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)
    

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()
    

# next run


In [ ]:
stat, struc_dict = cryspy_restart.restart()
jobs = Ctrl_job(stat, struc_dict)
jobs.check_job()
cwd_dict, struc_dict = handle_job_return_dict(jobs)


In [ ]:
jobs.id_queueing, jobs.id_running

In [ ]:
cwd_dict, struc_dict

In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()

In [ ]:
!verdi graph generate 14891

In [ ]:
!pwd